In [1]:
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoTokenizer
import pandas as pd

# 1️⃣ Read dataset
df = pd.read_csv("HEAR_Dataset.csv")  # adjust path if needed

# Extract texts
list_of_texts = df["Review Text"].tolist()

# Normalize labels: 1 = mentioned, 0 = not mentioned
binary_labels = df[["Pricing", "Appointments", "Medical Staff", "Customer Service", "Emergency Services"]]\
                  .applymap(lambda x: 0 if x==3 else 1).values

# 2️⃣ Split into train and test (80% / 20%)
texts_train, texts_test, labels_train, labels_test = train_test_split(
    list_of_texts, binary_labels, test_size=0.2, random_state=42
)

# 3️⃣ Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

# 4️⃣ Tokenize train and test separately
train_encodings = tokenizer(
    texts_train,
    truncation=True,
    padding=True,
    return_tensors="pt"
)

test_encodings = tokenizer(
    texts_test,
    truncation=True,
    padding=True,
    return_tensors="pt"
)

# 5️⃣ Convert labels to PyTorch tensors
labels_train_tensor = torch.tensor(labels_train, dtype=torch.float)
labels_test_tensor = torch.tensor(labels_test, dtype=torch.float)

# 6️⃣ Check shapes
print("Train input_ids shape:", train_encodings["input_ids"].shape)
print("Train labels shape:", labels_train_tensor.shape)
print("Test input_ids shape:", test_encodings["input_ids"].shape)
print("Test labels shape:", labels_test_tensor.shape)


/tmp/ipython-input-3093870351.py:14: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(lambda x: 0 if x==3 else 1).values
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Train input_ids shape: torch.Size([18403, 512])
Train labels shape: torch.Size([18403, 5])
Test input_ids shape: torch.Size([4601, 512])
Test labels shape: torch.Size([4601, 5])


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
from torch.utils.data import Dataset

class FeedbackDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Use tensor indexing directly
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]  # already tensor
        return item



# Create dataset objects
train_dataset = FeedbackDataset(train_encodings, labels_train_tensor)
test_dataset = FeedbackDataset(test_encodings, labels_test_tensor)


In [11]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

class MultiHeadBinaryClassifier(nn.Module):
    def __init__(self, bert_model_name="aubmindlab/bert-base-arabertv2", num_heads=5):
        super().__init__()
        self.bert = AutoModel.from_pretrained(bert_model_name)
        hidden_size = self.bert.config.hidden_size
        self.heads = nn.ModuleList([nn.Linear(hidden_size, 1) for _ in range(num_heads)])

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:,0,:]  # CLS token
        logits = [head(cls_embedding).squeeze(-1) for head in self.heads]
        return torch.stack(logits, dim=1)  # shape: [batch_size, 5]


In [12]:
model = MultiHeadBinaryClassifier()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCEWithLogitsLoss()
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [ ]:
from torch.utils.data import DataLoader
import torch.optim as optim

train_dataset = FeedbackDataset(train_encodings, labels_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

optimizer = optim.AdamW(model.parameters(), lr=2e-5)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(3):  # small number of epochs for prototype
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} loss: {total_loss/len(train_loader)}")


In [ ]:
model.eval()
with torch.no_grad():
    outputs = model(input_ids, attention_mask)  # shape [batch_size, 5]
    probs = torch.sigmoid(outputs)
    preds = (probs > 0.5).long()  # True/False per aspect